# Project Goals
The goal of this project is to perform a sentiment analysis on search results from Google recommendations, such as "Starbucks Oslo" or "Elkjøp Storo." The main idea is that relying solely on star ratings is an inconsistent way of measuring customer satisfaction for a given place. Therefore, I chose to implement sentiment analysis to get an idea of whether most reviews are, for example, "frustrated," "positive," or "negative."

For each search, the Google Places API will return up to five results—for instance, five Starbucks locations in Oslo. I will then perform a sentiment analysis on each place and conduct an overall NLP-based (Natural Language Processing) evaluation of all the places to decide on the best overall option.

---

# Approach

## Logical Flow
This project utilizes a sequential logical flow, resembling a pipeline or Chain of Thought (CoT).

We first start by gathering the place IDs (a unique identifier for each business location) and peripheral data (name and address) using the Google Places API. This will give us up to five relevant places based on our search term (such as "Starbucks Oslo").

Then, we obtain more data about each place using another Places API based on the place IDs. Here, we get access to the reviews, as well as ratings and additional peripheral data.

At this point, we are ready to start our NLP sequence.

- **Step One:** Loop through every place and every review for each place, writing a short sentiment summary of each review and labeling each review with single-word descriptors such as "happy," "frustrated," "dirty," etc. This is the first step in our CoT. We also assign a 0-10 rating to each review, where 0 is highly negative and 10 is highly positive.
- **Step Two:** Loop through every place and generate a short sentiment review, as well as sentiment labels for each place based on all the reviews for that place. This is the second step in our CoT and serves as a summarization of all the reviews for a single place. We then calculate the average rating for the place, which is done manually (i.e., outside the LLM).
- **Step Three:** Decide on the best place by evaluating each summary, the labels for each place, and the average rating. The LLM decides on the best place holistically, rather than simply selecting the one with the highest rating. The result is presented as a print statement containing a short summary of why this is the best place, along with the rating and labels.

---

## Code Structure
The code is structured in pairs of blocks. The first block contains the method definitions (e.g., `get_place_id(place_name)`), and the following block executes these methods with the necessary loops (e.g., `get_place_ids('Starbucks Oslo')`). The second block also prints the output.

- First, we gather the place IDs using `get_place_ids`.
- Then, we get the reviews using `get_reviews`.
- Next, we label each review with sentiments using `sentiment_grade_review` (this method makes a lot of API calls and takes a couple of minutes).
- After that, we perform sentiment grading for each place by summarizing the sentiments of each review using `make_overall_sentiment_rating`.
- Finally, we decide on the best place by comparing the sentiments of each place using `select_best_place`.

---

# Challenges

## API Costs and Speed
This pipeline makes a lot of API calls, with one run typically taking two to three minutes. This is both slow and expensive. A possible solution might be to use threading for improved performance and to reduce the amount of text being processed. For instance, one could try to select the best place by just looking at the labels, disregarding the summaries. Experimenting with lighter, cheaper models might also be a reasonable approach.

## Hallucinations
During testing, I observed that the average score of some places was negative—for example, -3.7 on a 0-10 scale. This is a misjudgment by the model, as it sometimes interprets -10 as highly negative, 0 as neutral, and 10 as positive, even though the instructions specify that 0 is negative and 10 is positive.

## Old Reviews
Reviews that are old (e.g., 3+ years) are not as relevant. Ideally, I should have filtered these out.

---

# Key Takeaways

## Vast Amounts of Unused NLP-Based Data
There is a lot of unstructured (natural language) data waiting to be exploited. Such data is often user-generated—in the form of reviews, complaints, comments, etc.—and is extremely valuable for the business since it comes directly from the users.

Other exciting applications may include advanced FAQ bots, sophisticated RAG-chains based on internal company data, and more. For an implementation of RAG, please try out my ChatGPT + Google Drive application, which creates an embedding store of your Google Drive on *drivesearcher.com*.

## Modern LLMs Are Incredibly Good at Context
The first LLMs, such as ChatGPT 3.0, were good at generating general text but struggled to understand user instructions (such as prompt tuning) and context. Modern LLMs, on the other hand, are highly adaptable and understand context in a meaningful way, making applications like the one showcased below much more useful.

## Chain of Thought
This script should have been built in LangChain or LangGraph, for that matter. As demonstrated by OpenAI, a chain of thought is a highly efficient way of enhancing a model's performance. Instead of using a single chat instance that tells the model, "There are 10 reviews for each place. Here are 5 places; select the best one," it is better to break up the process into a chain of thought, similar to how humans would process the data mentally. This is the approach I have taken:
- **First:** Mark the sentiment of each review.
- **Second:** Mark the sentiment of each place based on the reviews.
- **Third:** Decide on the best place based on sentiment.

For a production environment, I would definitely have used LangChain, as it simplifies typing and data management, but due to the extra setup overhead, I decided to skip it for now.

## Typing
My biggest challenge along the way was not understanding the JSON structure of the previous method. For instance, I spent a lot of time going back and forth trying to determine where the name of the place was stored. If I had used typing from the start, the development process would have been much easier and quicker.


---
# Using LLMs versus traditional approaches
As we can see in the bottom-most codeblock, based solely on star-rating one would want to go to Starbucks located in Nedre Storgate 6, 3015 Drammen, Norway. This is different from the models recomendation which (during this testrun) were  Starbucks at Jernbanetorget, siting its good customer service and convenient location.


In [184]:
#load env vars
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

PLACES_API_KEY = os.getenv("PLACES_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL")

In [185]:
#get the necesary data from the GOOGLE PLACES API
import requests

#use semantic search to get the PLACE_ID, then fetch the reviews

def get_place_ids(input_text='starbucks'):
    #fields = "name,rating,place_id"

    url = (
        f"https://maps.googleapis.com/maps/api/place/textsearch/json?"
        f"query={input_text}&key={PLACES_API_KEY}"
    )

    # Make the GET request
    response = requests.get(url)
    data = response.json()

    places = []
    for place in data.get("results", []):
        places.append({
            "name": place.get("name"),
            "rating": place.get("rating"),
            "place_id": place.get("place_id"),
            "address": place.get("formatted_address")
        })

    # # Print results
    # for p in places:
    #     print(f"Name: {p['name']}, Rating: {p['rating']}, Place ID: {p['place_id']}, Address: {p['address']}")

    return places

In [186]:
#retutns a list of {place_ids}
"""
[
  {
    "name": "string",
    "rating": float,
    "place_id": "string",
    "address": "string"
  },
  ...
]
"""
places = get_place_ids()
print(places)




[{'name': 'Starbucks', 'rating': 3.7, 'place_id': 'ChIJheGN9mFuQUYRuFHO03zsfts', 'address': 'Stenersgata 1, 0050 Oslo, Norway'}, {'name': 'Starbucks', 'rating': 4.1, 'place_id': 'ChIJN8KRC4FuQUYRDhgNKmK0H_E', 'address': 'Stranden 7, 0250 Oslo, Norway'}, {'name': 'Starbucks', 'rating': 3.8, 'place_id': 'ChIJvTPx3WNuQUYRCt_FaX_zYKs', 'address': 'Torggata 17B, 0183 Oslo, Norway'}, {'name': 'Starbucks', 'rating': 3.7, 'place_id': 'ChIJOT0lHIpuQUYRF6qkr3nXDeg', 'address': 'Jernbanetorget, 0154 Oslo, Norway'}, {'name': 'Starbucks Bogstadveien', 'rating': 4, 'place_id': 'ChIJNxeEQ9ptQUYRPvoxeV7OCpM', 'address': 'Bogstadveien 60A, 0366 Oslo, Norway'}, {'name': 'Starbucks Grønland', 'rating': 3.8, 'place_id': 'ChIJtyd-v19uQUYR464VTltSf5A', 'address': 'Grønlandsleiret 1, 0190 Oslo, Norway'}, {'name': 'Starbucks Strømmen', 'rating': 3.9, 'place_id': 'ChIJIfIRho97QUYRA1en-Jkua-U', 'address': 'Støperiveien 5, 2010 Strømmen, Norway'}, {'name': 'Starbucks', 'rating': 3.9, 'place_id': 'ChIJgyQFKmcTQUY

In [187]:
from urllib.parse import urlencode
#fetch the reviews based on place id

parameters = ['rating', 'reviews', 'types']
fields_str = ','.join(parameters) 

base_url = "https://maps.googleapis.com/maps/api/place/details/json"

def get_reviews(places :list):
    places_with_reviews = {}

    for place in places:
        place_id = place['place_id']
        name = place['name']
        address = place['address']
        rating = place['rating']

        query_params = {
            "fields": fields_str,
            "place_id": place_id,
            "key": PLACES_API_KEY
        }
        url = f"{base_url}?{urlencode(query_params)}"

        response = requests.get(url)
        data = response.json()

        #distill the necesary data
        result = data['result']
        reviews :list= result['reviews']

        #extract the necesary data from the reviews
        #that is: rating, text, time (maybe relative_time_description
        #add an ID (index) for easier managment
        formatted_reviews = []
        for i in range(len(reviews)):
            text = reviews[i]['text']
            formatted_reviews.append({
                "id": place_id,
                "index": i,
                "rating": rating,
                "text": text ,
                "name": name,
                "address": address
            })

        places_with_reviews[place_id]=formatted_reviews
    
    return places_with_reviews

In [188]:

"""
{
  "place_id_1": [
    {
      "id": "place_id_1",
      "index": int,
      "rating": int,
      "text": "string"
      "name": str    },
    ...
  ],
  "place_id_2": [
    {
      "id": "place_id_2",
      "index": int,
      "rating": int,
      "text": "string"
      "name" :str
    },
    ...
  ],
  ...
}
"""
places_with_reviews = get_reviews(places=places)
print(places_with_reviews)

{'ChIJheGN9mFuQUYRuFHO03zsfts': [{'id': 'ChIJheGN9mFuQUYRuFHO03zsfts', 'index': 0, 'rating': 3.7, 'text': 'It’s great to have Starbucks in the Oslo city mall. Unhappily the place is not well maintained. They used more than 15 mins to clean tables in a place with a lot of people coming and going.\n\nThe vanilla cream Frappuccino tasted fantastic according to the kid. The caramel pecan brownies was also fabulous.\n\nThe prices at the Norwegian Starbucks is way too high.', 'name': 'Starbucks', 'address': 'Stenersgata 1, 0050 Oslo, Norway'}, {'id': 'ChIJheGN9mFuQUYRuFHO03zsfts', 'index': 1, 'rating': 3.7, 'text': "Since I saw that a couple of girls already complained, I will also complain. An employee tried to steal my boyfriend. We went early in the morning for coffee and something sweet, and the girl (although it was clear that it was my boyfriend) was smiling the whole time and putting her heart on the glass for him, but not mine. I am very upset and angry and it hurts me when I see a w

In [189]:
#invoke the llm, prompt-tune it to become a sentiment analasysist
#i use langgraph as i have used this before
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

system_prompt = """
    Asses the sentiment of the review below.
    Make a short, couple sentence summary of the sentiment. 
    Add some single-word labels that describe teh sentiment.
    Add a score of 0-10 where 10 is highly positive sentiment, while 0 is highly negative"
"""

#summary: a short textual description of the sentiment
#label: a one-word description of the sentiment such as "happy", "sad", "disappointed", "delucional"
class Sentiment(BaseModel):
    summary: str
    labels: list[str]
    score: int

#returns a set of reviews
def sentiment_grade_review(reviews :list):

    #loop throiugh each review and store the data in llm_responses
    llm_responses = []

    for review in reviews:
        place_id = review['id']
        index = review['index']
        rating = review['rating']
        text = review['text']
        name = review['name']
        address = review['address']

        print(f"working on review {index+1}/{len(reviews)}")

        #skip the reviews with no text
        if not text:
            pass

        completion = client.beta.chat.completions.parse(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": text},
            ],
            response_format=Sentiment,
        )
        event = completion.choices[0].message.parsed

        llm_responses.append({
            "id": place_id,
            "index": index,
            "name": name,
            "address": address,
            "rating": rating,
            "text": text,
            "summary": event.summary,
            "labels": event.labels,
            "score": event.score
        })
    
    return llm_responses




In [190]:




"""
{
  "place_id_1": [
    {
      "id": "place_id_1",
      "index": int,
      "rating": int,
      "text": "string",
      "summary": "string",
      "name": str
      "labels": ["string", ...],
      "score": int
    },
    ...
  ],
  "place_id_2": [
    {
      "id": "place_id_2",
      "index": int,
      "rating": int,
      "text": "string",
      "summary": "string",
      "name": str
      "labels": ["string", ...],
      "score": int
    },
    ...
  ]
}
"""
places_with_sentiment = {}
for index, place_id in enumerate(list(places_with_reviews.keys())):  
    print(f"\n Working on place {index+1}/{len(places_with_reviews.keys())}")
    reviews = places_with_reviews[place_id] #list of reviews

    sentiments =  sentiment_grade_review(reviews=reviews)
    places_with_sentiment[place_id] = sentiments

print(places_with_sentiment)




 Working on place 1/9
working on review 1/5
working on review 2/5
working on review 3/5
working on review 4/5
working on review 5/5

 Working on place 2/9
working on review 1/5
working on review 2/5
working on review 3/5
working on review 4/5
working on review 5/5

 Working on place 3/9
working on review 1/5
working on review 2/5
working on review 3/5
working on review 4/5
working on review 5/5

 Working on place 4/9
working on review 1/5
working on review 2/5
working on review 3/5
working on review 4/5
working on review 5/5

 Working on place 5/9
working on review 1/5
working on review 2/5
working on review 3/5
working on review 4/5
working on review 5/5

 Working on place 6/9
working on review 1/5
working on review 2/5
working on review 3/5
working on review 4/5
working on review 5/5

 Working on place 7/9
working on review 1/5
working on review 2/5
working on review 3/5
working on review 4/5
working on review 5/5

 Working on place 8/9
working on review 1/5
working on review 2/5
wo

In [191]:
system_prompt = f"""
    Based on the labels and summaries below you are to make an overall sentimentanalysis.
    Write a short summary describing the general sentiment.
    Add the most common sentiment-labels
"""

class OverallSentiment(BaseModel):
    summary: str
    labels: list[str]


def make_overall_sentiment_rating(sentiments :list):
    number_of_reviews = len(sentiments)
    total_score = 0
    summaries = ''
    total_labels = []

    for s in sentiments:
        # place_id = s['id']
        # index = s['index']
        # rating = s['rating']
        # text = s['text']
        summary = s['summary']
        labels = s['labels']
        score = s['score']

        total_score += score
        summaries += "\n\n\n" + summary
        total_labels.extend(labels)

    avg_score = round(total_score / number_of_reviews, 1)

    prompt = summaries + "\nLabels: " + ", ".join(total_labels)

    #pass to llm
    completion = client.beta.chat.completions.parse(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ],
        response_format=OverallSentiment,
    )
    event = completion.choices[0].message.parsed

    summary = event.summary
    labels = event.labels

    summarised_review = {
        "name": sentiments[0]['name'], #extraxt the name and adr from the first review
        "address": sentiments[0]['address'],
        "summary": summary,
        "labels": labels,
        "avg_score": avg_score,
    }

    return summarised_review
        

In [ ]:
"""
{
  "place_id": {
    "name": str
    "summary": "Short textual summary of the overall sentiment.",
    "labels": [
      "label1",
      "label2",
      "label3"
    ],
    "avg_score": float
  }
}
"""
summarised_reviews = {}
for index, (place_id, sentiment) in enumerate(places_with_sentiment.items(), start=1):
  print(f"workin on summarising reviews for place {index}/{len(places_with_sentiment.keys())}")
  pws = places_with_sentiment[place_id]
  summarised_review = make_overall_sentiment_rating(sentiments=pws)
  summarised_reviews[place_id] = summarised_review

print(summarised_reviews)

workin on summarising reviews for place 1/9
workin on summarising reviews for place 2/9
workin on summarising reviews for place 3/9


In [ ]:
system_prompt = f"""
    Based on the sentiment summaries, labels and average score below you are to decide on the best place. Write a short summary about why this is the best place. 
    In the summary, include the average rating. Add labels describing the place.
"""


class Bestplace(BaseModel):
    summary: str
    labels: list[str]

"""
{
  "place_id": {
    "summary": "Short textual summary of the overall sentiment.",
    "name":str
    "labels": [
      "label1",
      "label2",
      "label3"
    ],
    "avg_score": float
  }
}
"""

def select_best_place(summarised_reviews :dict):
    
    prompt = ''
    for place_id in summarised_reviews.keys():
        place = summarised_reviews[place_id]
        summary = place['summary']
        labels = place['labels'] #list
        name = place['name']
        address = place['address']
        avg_score = place['avg_score']

        prompt += f"\n\n\n Name of the place: {name} \n Address: {address} \n Average 0-10 score, where 10 is highly positive is {avg_score} \n Summary: {summary} \n Labels: {labels}"

    completion = client.beta.chat.completions.parse(
    model=OPENAI_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ],
    response_format=OverallSentiment,
    )

    event = completion.choices[0].message.parsed
    return event

In [ ]:
best_place = select_best_place(summarised_reviews=summarised_reviews)
summary = best_place.summary
labels = best_place.labels

print(f"""
{summary}
\n\n
This place has the following sentiment labels:\n
{", ".join(labels)}
      """)



Starbucks at Jernbanetorget stands out as the best option with an impressive rating of 8.4. Customers praise its friendly service, high-quality offerings, and pleasant experiences, making it a top choice despite minor inconveniences.



This place has the following sentiment labels:

positive, pleasant, friendly, happy, convenient
      


In [ ]:
#Comapare my results to just the star-rating
places = get_place_ids() #starbucks oslo
best_place = None
best_rating = 0

for place in places:
    rating = place['rating']
    if rating>best_rating:
        best_place=place
        best_rating=rating

print(f"""
The best place based solely on star-rating is {place['name']} located in {place['address']}.
This place has a star-rating of {best_rating}
\n\n
Compared to our sentiment model: \n {summary}
      """)


The best place based solely on star-rating is Starbucks located in Nedre Storgate 6, 3015 Drammen, Norway.
This place has a star-rating of 4.1



Compared to our sentiment model: 
 Starbucks at Jernbanetorget stands out as the best option with an impressive rating of 8.4. Customers praise its friendly service, high-quality offerings, and pleasant experiences, making it a top choice despite minor inconveniences.
      
